<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/gpt2_small_1_N(MSR%2BDIV%2BJUL)_nofunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install datasets

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cwe file/MSR+julite+Div_final.csv')

In [6]:
df

,CWE ID,vul,lang,code,labels
0,safe,0,c,void func(struct net_device *d) { isdn_net_loc...,0
1,safe,0,c,void func(isdn_net_local *lp) { struct sk_buff...,0
2,safe,0,c,static __inline__ void func(isdn_net_local *lp...,0
3,safe,0,c,"void func(char *name, struct net_device *maste...",0
4,safe,0,c,void func(char *name) { u_long flags; isdn_net...,0
...,...,...,...,...,...
15833,safe,0,c,\nvoid func(char * data);\n\nstatic void func(...,0
15834,safe,0,cpp,\nstatic void func()\n{\n long * data;\n ...,0
15835,safe,0,cpp,\nstatic void func()\n{\n int64_t * data;\n...,0
15836,safe,0,c,static const void func(OnigEncoding mbctype) {...,0


In [7]:
df.labels.value_counts()

labels
1    2800
0    2700
2    1978
3    1681
4    1536
5    1380
6    1059
7    1057
8     912
9     735
Name: count, dtype: int64

In [8]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [31]:
model = AutoModelForSequenceClassification.from_pretrained("ComCom/gpt2-small", num_labels = 10)
tokenizer = AutoTokenizer.from_pretrained("ComCom/gpt2-small")

# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

batch_size = 16
epoch_num = 1000
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ComCom/gpt2-small and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    df['code'] = df['code'].replace(re.compile(r'#include .*?\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#include "*.*?\n'), "", regex=True)
    # 매크로 정의 제거
    df['code'] = df['code'].replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 전처리 지시문 제거
    df['code'] = df['code'].replace(re.compile(r'#undef\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#if\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#else\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#elif\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#endif'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    df['code'] = df['code'].replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    df['code'] = df['code'].replace(re.compile(r'\n\s*\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return*.*?;'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return;'), "", regex=True)
    # 데이터프레임을 CSV 파일로 저장
    df.to_csv(file_name, index=False)
    return df  # 수정된 데이터프레임 반환

def tokenized(examples):
    return tokenizer(examples['code'], pad_to_max_length=True, max_length=MAX_LEN, truncation=True, return_token_type_ids=True)


In [33]:
preprocess(df, "preprocess.csv")
dataset = load_dataset("csv", data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized, remove_columns=['code'], batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/15838 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [34]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=100)

In [35]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [36]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [37]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics

In [38]:
from transformers import TrainerCallback
class CustomTrainerCallback(TrainerCallback):
    def __init__(self, trainer, eval_dataset, early_stopping_patience=5):
        self.trainer = trainer
        self.eval_dataset = eval_dataset
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = float('inf')
        self.patience_counter = 0
        self.incorrect_samples = []

    def on_evaluate(self, args, state, control, **kwargs):
        eval_loss = kwargs.get('metrics', {}).get('eval_loss')

        if eval_loss is not None:
            if eval_loss < self.best_loss:
                self.best_loss = eval_loss
                self.patience_counter = 0
            else:
                self.patience_counter += 1

            if self.patience_counter >= self.early_stopping_patience:
                print(f"No improvement in {self.early_stopping_patience} evaluation steps. Stopping training early.")
                control.should_training_stop = True

        # Evaluate를 위한 DataLoader 설정 및 평가 로직
        eval_dataloader = self.trainer.get_eval_dataloader(self.eval_dataset)
        self.trainer.model.eval()
        for batch in eval_dataloader:
            batch = {k: v.to(self.trainer.args.device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = self.trainer.model(**batch)
                predictions = torch.argmax(outputs.logits, dim=-1)
                labels = batch['labels']
                incorrect_mask = predictions != labels

            if incorrect_mask.any():
                incorrect_samples = {k: v[incorrect_mask].cpu().numpy() for k, v in batch.items()}
                self.incorrect_samples.append(incorrect_samples)

        print("Collected incorrect predictions for further analysis.")

In [39]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1000,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)
callback = CustomTrainerCallback(encoded_dataset['test'], 5)

# 트레이너 초기화 및 사용자 정의 콜백 추가

# 콜백 인스턴스 생성 및 트레이너 초기화

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics
)
callback = CustomTrainerCallback(trainer, encoded_dataset['test'], 5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    callbacks=[callback]
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9
500,2.193000,1.834270,0.336490,0.336490,0.272398,0.227165,0.336490,0.415049,0.307692,0.086697,0.299139,0.269027,0.852259,0.006390,0.026738,0.000000,0.008658
1000,1.653300,1.470482,0.492003,0.492003,0.466219,0.415999,0.492003,0.691552,0.457119,0.351240,0.465823,0.477449,0.929016,0.188034,0.249377,0.007353,0.343023
1500,1.273100,1.191945,0.600168,0.600168,0.589874,0.566261,0.600168,0.725530,0.543990,0.550000,0.552017,0.537597,0.927961,0.529070,0.492035,0.318538,0.485876
2000,0.983900,1.145198,0.647306,0.647306,0.642542,0.623664,0.647306,0.783405,0.591404,0.589942,0.565737,0.635036,0.916667,0.624782,0.552486,0.460554,0.516624
2500,0.794100,1.150049,0.661406,0.661406,0.656538,0.637462,0.661406,0.799325,0.606345,0.610199,0.619748,0.620447,0.915493,0.591940,0.527514,0.520161,0.563452
3000,0.587500,1.141711,0.693813,0.693813,0.692003,0.674081,0.693813,0.840937,0.650267,0.627329,0.643462,0.662198,0.904018,0.645161,0.607242,0.555118,0.605081
3500,0.451800,1.101834,0.704545,0.704545,0.703655,0.686609,0.704545,0.838669,0.660333,0.661276,0.657845,0.663765,0.905028,0.675241,0.625624,0.585714,0.592593
4000,0.346800,1.177579,0.715067,0.715067,0.713271,0.701488,0.715067,0.824009,0.668908,0.671506,0.647937,0.683805,0.931315,0.707531,0.647934,0.583658,0.648276
4500,0.276700,1.250441,0.717803,0.717803,0.715262,0.705760,0.717803,0.817617,0.660079,0.669499,0.671815,0.682379,0.935218,0.711111,0.664723,0.585821,0.659341


Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
Collected incorrect predictions for further analysis.
